In [1]:
# Initial imports.
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Initial Trial with csv files

In [2]:
# from google.colab import files 
# uploaded = files.upload()

In [3]:
# import io
# df = pd.read_csv(io.BytesIO(uploaded['ml_BattingFielding.csv']))
# df.head()
# # Dataset is now stored in a Pandas Dataframe

# Database Connection

In [4]:
# dependencies
import psycopg2
from config import config

In [5]:
# get parameters
params = config()

In [6]:
# connect to PostgreSQL database
conn = psycopg2.connect(**params)

# create a new cursor
cur = conn.cursor()

In [7]:
# funcion to create a pandas df given parameters of: 1. SQL query 2. database connection
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [8]:
# trial with only 1 table
batting_df = create_pandas_table('SELECT * FROM "Batting"')
batting_df.head()

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,AVG
0,aardsda01,331,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,aaronha01,3298,12364,2174,3771,624,98,755,2297.0,240.0,73.0,1402,1383.0,293.0,32.0,21.0,121.0,328.0,0.304998
2,aaronto01,437,944,102,216,42,6,13,94.0,9.0,8.0,86,145.0,3.0,0.0,9.0,6.0,36.0,0.228814
3,aasedo01,448,5,0,0,0,0,0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,abadan01,15,21,1,2,0,0,0,0.0,0.0,1.0,4,5.0,0.0,0.0,0.0,0.0,1.0,0.095238


### Build batting and fielding dataframe

In [34]:
# build batting/fielding df for machine learning analysis with SQL JOIN  multiple tables
combined_df = create_pandas_table('SELECT h."playerID",h."inducted", b."R",b."H",b."2B",b."3B",b."HR",b."RBI",b."SB", \
                                b."CS",b."BB",b."SO",b."IBB",b."SH",b."SF",b."GIDP",b."AVG",f."PO",f."A",f."E",f."DP",a."award_count" \
                                   FROM "Hall_Of_Fame" AS h \
                                   INNER JOIN "Batting" as b ON (h."playerID" = b."playerID") \
                                   INNER JOIN "Fielding" as f ON (h."playerID" = f."playerID") \
                                   INNER JOIN "Awards" as a ON (h."playerID" = a."playerID")')
combined_df.head()

,playerID,inducted,R,H,2B,3B,HR,RBI,SB,CS,...,IBB,SH,SF,GIDP,AVG,PO,A,E,DP,award_count
0,aaronha01,Y,2174,3771,624,98,755,2297.0,240.0,73.0,...,293.0,21.0,121.0,328.0,0.304998,7436,429,144.0,218,16
1,abbotji01,N,0,2,0,0,0,3.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.095238,72,300,9.0,16,2
2,adamsba01,N,79,216,31,15,3,75.0,1.0,1.0,...,0.0,35.0,0.0,0.0,0.211973,58,648,17.0,17,2
3,ageeto01,N,558,999,170,27,130,433.0,167.0,81.0,...,26.0,21.0,15.0,99.0,0.255368,2371,53,61.0,18,3
4,akerja01,N,3,7,2,0,0,4.0,0.0,0.0,...,0.0,10.0,0.0,1.0,0.076087,65,167,8.0,20,1


In [10]:
combined_df.shape

(751, 22)

In [11]:
combined_df.isnull().sum()

playerID       0
inducted       0
R              0
H              0
2B             0
3B             0
HR             0
RBI            0
SB             0
CS             0
BB             0
SO             0
IBB            0
SH             0
SF             0
GIDP           0
AVG            5
PO             0
A              0
E              0
DP             0
award_count    0
dtype: int64

In [12]:
# drop the rows with null values
combined_df = combined_df.dropna(subset=['AVG'])

In [13]:
# double check df shape
combined_df.shape

(746, 22)

### Build pitching dataframe

In [14]:
# build pitching df for machine learning analysis with SQL JOIN multiple tables
pitching_df = create_pandas_table('SELECT h."playerID",h."inducted",p."W",p."L",p."CG",p."SHO",p."SV",p."H_allowed",p."ER",p."HR_allowed",p."BB_allowed",p."StruckOut",p."BAOpp",p."ERA",p."WP",p."HitBatsmen",p."BK",p."R_allowed",p."SH_allowed",p."SF_allowed",p."GIDP_pitcher",a."award_count" FROM "Hall_Of_Fame" AS h INNER JOIN "Pitching" as p ON (h."playerID" = p."playerID") INNER JOIN "Awards" as a ON (h."playerID" = a."playerID")')
pitching_df.head()

,playerID,inducted,W,L,CG,SHO,SV,H_allowed,ER,HR_allowed,...,BAOpp,ERA,WP,HitBatsmen,BK,R_allowed,SH_allowed,SF_allowed,GIDP_pitcher,award_count
0,abbotji01,N,87,108,31,6,0,1779,791,154,...,0.280364,4.496364,53,32.0,11,880,70.0,47.0,200.0,2
1,adamsba01,N,194,140,206,44,15,2841,917,68,...,0.256667,3.733684,26,47.0,2,1129,0.0,0.0,0.0,2
2,akerja01,N,47,45,0,0,123,679,272,64,...,0.253571,3.854286,13,40.0,0,312,32.0,12.0,19.0,1
3,alexape01,Y,373,208,437,90,32,4868,1476,164,...,0.253810,2.972381,39,70.0,1,1851,0.0,0.0,0.0,21
4,allenjo02,N,142,75,109,17,18,1849,813,104,...,0.244667,3.765333,68,38.0,5,924,0.0,0.0,0.0,4


In [15]:
# check shape of df
pitching_df.shape

(284, 22)

In [16]:
# check for null values
pitching_df.isnull().sum()

playerID         0
inducted         0
W                0
L                0
CG               0
SHO              0
SV               0
H_allowed        0
ER               0
HR_allowed       0
BB_allowed       0
StruckOut        0
BAOpp           14
ERA              2
WP               0
HitBatsmen       0
BK               0
R_allowed        0
SH_allowed       0
SF_allowed       0
GIDP_pitcher     0
award_count      0
dtype: int64

In [17]:
# drop the rows with null values
pitching_df = pitching_df.dropna(subset=['ERA','BAOpp'])

In [18]:
# double check df shape
pitching_df.shape

(268, 22)

# Machine Learning

In [19]:
# define which dataframe will be used in machine learning model
# e.g. pitching or batting/fielding

df = combined_df

In [20]:
# encode inducted column from Y/N to numeric 0/1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
df2['inducted'] = le.fit_transform(df2['inducted'])

In [21]:
# encode playerID to numeric
# possibly remove if we want to keep playerID's as index to later reference player names

# df2['playerID'] = le.fit_transform(df2['playerID'])

In [22]:
# define input (X) and output (y) variables
# drop columns of unnecessary features in X 
# batting/fielding 1st iteration showed the following features contributed <3%:
# SF, CS, IBB, SB

y = df2["inducted"]
X = df2.drop(columns=["inducted", "playerID", "SF", "CS", "IBB", "SB"])

In [23]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [24]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [26]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [27]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [28]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,127,12
Actual 1,13,35


In [29]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [30]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,127,12
Actual 1,13,35


Accuracy Score : 0.8663101604278075
Classification Report
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       139
           1       0.74      0.73      0.74        48

    accuracy                           0.87       187
   macro avg       0.83      0.82      0.82       187
weighted avg       0.87      0.87      0.87       187



In [31]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.09405157, 0.09031537, 0.04394405, 0.04595591, 0.03805029,
       0.05852511, 0.04295533, 0.0558363 , 0.04382171, 0.03826045,
       0.09112769, 0.04618862, 0.06312932, 0.04817306, 0.03818609,
       0.16147912])

In [32]:
# sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.16147912339241308, 'award_count'),
 (0.09405157260170642, 'R'),
 (0.09112768741100967, 'AVG'),
 (0.09031537296116243, 'H'),
 (0.06312932357843094, 'A'),
 (0.05852511210356192, 'RBI'),
 (0.055836295695513474, 'SO'),
 (0.048173059370415085, 'E'),
 (0.04618862487663315, 'PO'),
 (0.04595590841272363, '3B'),
 (0.0439440495451252, '2B'),
 (0.043821710445384736, 'SH'),
 (0.04295532881342847, 'BB'),
 (0.03826045232174703, 'GIDP'),
 (0.03818608929511483, 'DP'),
 (0.03805028917562995, 'HR')]

In [33]:
rf_model.feature_importances_.sum()

1.0